In [1]:
# %pip install langchain
# %pip install langchain-community
# %pip install pypdf
# %pip install unstructured
# %pip install openpyxl
# %pip install --upgrade --quiet  gpt4all > /dev/null
# %pip install --upgrade --quiet langchain-elasticsearch langchain-openai tiktoken langchain
# %pip install python-dotenv

## Loading Documents

In [1]:
import os
import pandas as pd
from langchain.document_loaders import PyPDFLoader, CSVLoader
from langchain_community.document_loaders import UnstructuredExcelLoader
from langchain.docstore.document import Document
import csv


In [3]:
def convert_excel_to_csv(directory):
    # Get a list of all Excel files in the directory
    excel_files = [file for file in os.listdir(directory) if file.endswith(('.xls', '.xlsx'))]
    for excel_file in excel_files:
        # Define the path to the Excel file
        excel_file_path = os.path.join(directory, excel_file)
        
        # Read all sheets from the Excel file
        excel_data = pd.read_excel(excel_file_path, sheet_name=None)

        # Process each sheet separately
        for sheet_name, df in excel_data.items():
            # Define the output CSV file path, including the sheet name
            output_csv_file = os.path.join(directory, f"{os.path.splitext(excel_file)[0]}_{sheet_name}.csv")

            # Drop unnamed columns
            df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
            
            # Drop rows where all values are NaN or columns that are entirely NaN
            df.dropna(how='all', inplace=True)
            df.dropna(axis=1, how='all', inplace=True)
            
            # Save the DataFrame to a CSV file
            df.to_csv(output_csv_file, index=False, encoding='utf-8')
            
            print(f"Converted {excel_file} - Sheet: {sheet_name} to {output_csv_file}")

In [2]:
directory = "KnowledgeBase"

In [4]:

# convert_excel_to_csv(directory)

In [13]:

allDocs = {}

for filename in os.listdir(directory):
    if filename.endswith(".pdf"):
        print("Processing", filename)
        pdfLoader = PyPDFLoader(os.path.join(directory, filename))

        if "pdfs" not in allDocs:
            allDocs["pdfs"] = []

        allDocs["pdfs"].extend(pdfLoader.load())

    elif filename.endswith(".xlsx"):
        print("Processing", filename)
        excelLoader = UnstructuredExcelLoader(os.path.join(directory, filename))


        if "xlsx" not in allDocs:
            allDocs["xlsx"] = []

        allDocs["xlsx"].extend(excelLoader.load())

    # elif filename.endswith(".csv"):
    #     print("Processing", filename)
    #     csvloader = CSVLoader(os.path.join(directory, filename))

    #     if "csv" not in allDocs:
    #         allDocs["csv"] = []

    #     allDocs["csv"].extend(csvloader.load())
    else:
        continue

Processing Risikoanalyse.xlsx


/Users/I748655/Library/Python/3.9/lib/python/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


Processing A18_Liste_der_Dienstleister.xlsx
Processing Zuordnung_Prozesse_Standort.xlsx
Processing A21_Definition_Schutzbedarfskategorien.pdf
Processing Ergebnis_des_IT-Grundschutz-Checks.xlsx
Processing A05_Richtlinie_Lenkung_Korrektur_Vorbeugungsm.pdf
Processing Nicht_modellierte_Bausteine.xlsx
Processing Beschreibung_Recplast.pdf
Processing A3_Modellierung_Recplast_GmbH.xlsx
Processing A04_Richtlinie_internen_ISMS_Auditierung.pdf
Processing A11_Abgrenzung_des_Informationsverbunds.pdf
Processing A03_Richtlinie_Lenkung_Dokumenten_und_Aufzeichnungen.pdf
Processing A02_Richtlinie_Risikoanalyse.pdf
Processing A01_Sicherheitsleitlinie.pdf
Processing Risikoanalyse_GP_Ebene.xlsx
Processing A1_Strukturanalyse_RECPLAST_GmbH.xlsx


/Users/I748655/Library/Python/3.9/lib/python/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


Processing Schutzbedarfsfeststellung.xlsx
Processing Realisierungsplan.xlsx
Processing A1_Strukturanalyse-Abhaengigkeiten_RECPLAST_GmbH.xlsx


In [6]:
# To convert csv file to Langchain doc
def process_csv(file_path):
    docs = []
    with open(file_path, newline="", encoding='utf-8-sig') as csvfile:
        csv_reader = csv.DictReader(csvfile)
        columns = csv_reader.fieldnames  # Get the column names dynamically
        for i, row in enumerate(csv_reader):
            to_metadata = {col: row[col] for col in columns if col in row}
            values_to_embed = {k: row[k] for k in columns if k in row}
            to_embed = "\n".join(f"{k.strip()}: {v.strip()}" for k, v in values_to_embed.items())
            newDoc = Document(page_content=to_embed, metadata=to_metadata)
            docs.append(newDoc)
    return docs

In [ ]:
# allDocs['csv'] = []
# for filename in os.listdir(directory):
#     if filename.endswith('.csv'):
#         file_path = os.path.join(directory, filename)
#         allDocs['csv'].extend(process_csv(file_path))

In [14]:
print("Loaded", len(allDocs["pdfs"]), "pdf pages")
print("Loaded", len(allDocs["xlsx"]), "excel sheets")
# print("Loaded", len(allDocs["csv"]), "CSV rows")


Loaded 129 pdf pages
Loaded 11 excel sheets


In [15]:
print("First excel page content: \n", allDocs["xlsx"][0].page_content[0:1000])
# print("CSV row: \n", allDocs["csv"][400].page_content)

First excel page content: 
 


Zielobjekt
Gefährdung
Eintrittshäufigkeit ohne zusätzliche Maßnahmen
Auswirkung ohne zusätzliche Maßnahmen
Risiko ohne zusätzliche Maßnahmen
Risikobehandlungs-option
Erläuterung zur Risikobehandlung
Eintrittshäufigkeit mit zusätzlichen Maßnahmen
Auswirkung mit zusätzlichen Maßnahmen
Risiko mit zusätzlichen Maßnahmen


A002
G 0.11 Ausfall oder Störung von Dienstleistern
selten
vernachlässigbar
gering
Risikoakzeptanz
Aufgrund der bereits umgesetzten Maßnahmen wird das Risiko als vertretbar eingeschätzt.





A002
G 0.14 Ausspähen von Informationen (Spionage)
selten
begrenzt
gering
Risikoakzeptanz
Aufgrund der bereits umgesetzten Maßnahmen wird das Risiko als vertretbar eingeschätzt.





A002
G 0.15 Abhören
selten
beträchtlich
mittel
Risikoakzeptanz
Aufgrund der bereits umgesetzten Maßnahmen wird das Risiko als vertretbar eingeschätzt.





A002
G 0.16 Diebstahl von Geräten, Datenträgern oder Dokumenten
selten
begrenzt
gering
Risikoakzeptanz
Aufgrund der be

In [16]:
print("First pdf page content: \n", allDocs["pdfs"][0].page_content[0:1000])

First pdf page content: 
 Definition der 
Schutzbedarfskategorien
Seite 1 von 8



In [18]:
print("First excel page metadata:", allDocs["xlsx"][0].metadata)

First excel page metadata: {'source': 'KnowledgeBase/Risikoanalyse.xlsx'}


In [19]:
print("First pdf page metadata:", allDocs["pdfs"][78].metadata)

First pdf page metadata: {'source': 'KnowledgeBase/Beschreibung_Recplast.pdf', 'page': 63}


## Splitting Documents

In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter

r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=150,
    chunk_overlap=0,
    separators=["\n\n", "\n", "(?<=\. )", " ", ""]
)

r_splitter_excel = RecursiveCharacterTextSplitter(
    chunk_size=150,
    chunk_overlap=0,
    separators=["\r\n", "\n", "\t", ",", " "]
)
# splitter = CharacterTextSplitter(
#     separator="\n",
#     chunk_size=500, 
#     chunk_overlap=0,
#     length_function=len
#     )


In [20]:
splits={}

for key in allDocs:
    splits[key] = r_splitter.split_documents(allDocs[key])

In [21]:
for key in allDocs:
    print("Number of splits for", key, ":", len(splits[key]))

Number of splits for xlsx : 15917
Number of splits for pdfs : 1790


In [22]:
print("First 5 PDF splits: ", splits["pdfs"][:5])

First 5 PDF splits:  [Document(page_content='Definition der \nSchutzbedarfskategorien\nSeite 1 von 8', metadata={'source': 'KnowledgeBase/A21_Definition_Schutzbedarfskategorien.pdf', 'page': 0}), Document(page_content='Dokumenteneigenschaften\nVerantwortung Informationssicherheitsbeauftragter\nKlassifizierung S2 intern\nGültigkeitszeit Unbegrenzt', metadata={'source': 'KnowledgeBase/A21_Definition_Schutzbedarfskategorien.pdf', 'page': 1}), Document(page_content='Überarbeitungsintervall Jährlich\nNächste Überarbeitung Oktober 2020\nDateiname A.2.1_Definition_Schutzbedarfskategorien\nDokumentenstatus und Freigabe', metadata={'source': 'KnowledgeBase/A21_Definition_Schutzbedarfskategorien.pdf', 'page': 1}), Document(page_content='Status Version Datum Name und Abteilung/Firma\nErstellt 1.0 tt.mm.jjjj\nDokumentenhistorie\nVersion Änderung Datum Autor\n1.0 tt.mm.jjjj\nSeite 2 von 8', metadata={'source': 'KnowledgeBase/A21_Definition_Schutzbedarfskategorien.pdf', 'page': 1}), Document(page_co

In [23]:
print("First 5 Excel splits", splits["xlsx"][:5])
# # print("First 5 CSV splits", splits["csv"][:5])
# splits["csv"][0].page_content


First 5 Excel splits [Document(page_content='Zielobjekt\nGefährdung\nEintrittshäufigkeit ohne zusätzliche Maßnahmen\nAuswirkung ohne zusätzliche Maßnahmen\nRisiko ohne zusätzliche Maßnahmen', metadata={'source': 'KnowledgeBase/Risikoanalyse.xlsx'}), Document(page_content='Risikobehandlungs-option\nErläuterung zur Risikobehandlung\nEintrittshäufigkeit mit zusätzlichen Maßnahmen\nAuswirkung mit zusätzlichen Maßnahmen', metadata={'source': 'KnowledgeBase/Risikoanalyse.xlsx'}), Document(page_content='Risiko mit zusätzlichen Maßnahmen', metadata={'source': 'KnowledgeBase/Risikoanalyse.xlsx'}), Document(page_content='A002\nG 0.11 Ausfall oder Störung von Dienstleistern\nselten\nvernachlässigbar\ngering\nRisikoakzeptanz', metadata={'source': 'KnowledgeBase/Risikoanalyse.xlsx'}), Document(page_content='Aufgrund der bereits umgesetzten Maßnahmen wird das Risiko als vertretbar eingeschätzt.', metadata={'source': 'KnowledgeBase/Risikoanalyse.xlsx'})]


## Create Embeddings

In [24]:
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_elasticsearch import ElasticsearchStore
import numpy as np
from dotenv import load_dotenv
import os

In [25]:
model_name = "all-MiniLM-L6-v2.gguf2.f16.gguf"

In [26]:
embedding = GPT4AllEmbeddings(model_name=model_name)

In [27]:
sentence1 = "i like dogs"
sentence2 = "i like canines"
sentence3 = "the weather is ugly outside"

In [28]:
embedding1 = embedding.embed_query(sentence1)
embedding2 = embedding.embed_query(sentence2)
embedding3 = embedding.embed_query(sentence3)

In [29]:
load_dotenv()  # This loads the .env file at the application start
password = os.getenv('passwd')
api_key = os.getenv('api_key')

In [30]:
cloud_id  = '01a9e8bc7d7e4b91affdfcc8b88e70dd:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvJDVjZDFmMGQ0NDJkMzQ3ODA5ZmNiMjk4MTM5NmE4NGMxJGRmNTZhYTExYmE2YzRlZWZhOTE4NTBkMDJjZTY2MDIx'
elastic_vector_search = ElasticsearchStore(
    es_cloud_id=cloud_id,
    index_name="embeddings_index",
    embedding=embedding,
    es_user="group13",
    es_password=password,
    es_api_key=api_key
)

In [24]:
# elastic_vector_search.delete("embeddings_index")

True

In [31]:
elastic_vector_search.add_documents(splits["pdfs"])

['52d097bf-08c2-46ba-9c62-1ed0ed87ddb5',
 '4a70a355-f318-4aba-abba-e1bc156d87e5',
 'deda9b38-cd3a-4a94-a613-7c122d36dbd6',
 'ecb1607b-8623-416d-beb7-b8c2da3dfd84',
 '8a083c8c-befb-42fd-b5fa-ea978efed43b',
 'b9c27307-c880-4fd4-9824-a045aa09fdce',
 '429f9c3d-00c4-4555-aba3-0c6c1783c60d',
 '233e31e7-d9c3-468c-a156-4b7549850fdf',
 '0a0e990c-0526-477a-9ef5-63dbe4deb1bd',
 'f1667757-cf4b-4ea7-817a-8d8947fd7e92',
 '46882f90-6410-463c-b1b8-95bdea1fa950',
 'cdbf5e10-cad3-4e9b-98d5-a885e3fe10f3',
 '0c17a4cf-668f-4752-9e5f-dfa536bed932',
 'e8985cdf-95ed-44c2-83dc-6352bec85669',
 '2f715f66-8af1-4cd5-8c84-c8b461ce944e',
 '86582c98-2744-4cee-bb91-191694d8d56d',
 '1201576a-ed7c-4b38-b9b9-2a553e1a3db5',
 'fd1fde52-4ddb-4627-8693-3f1519355eb1',
 '9862e8b8-8547-4ce1-a60a-cd55cce4786c',
 'f2cef3f9-9104-4234-9c3f-a3a85a9e12ae',
 '7bba239a-917a-4798-bc63-4d45013bd4ee',
 '01ad0101-de8f-4973-856e-219c3dbc4c11',
 '8969a341-153a-4fc9-bc16-b09230bcd065',
 '90e2a44c-5b5f-4c73-b3ad-ad837651a033',
 '9d2f4df2-bad8-

In [32]:
elastic_vector_search.add_documents(splits["xlsx"])

['b7ccafda-61d2-491b-991f-367d79650d16',
 '7deb01ec-9d13-4bdb-bc70-9655939d9eee',
 'bdb58e80-e3e3-44a6-aa43-2ed215cb024f',
 'a1aa2cb2-6cf0-4e6f-8448-d733a724e851',
 '8ea1f39c-147f-4ad4-b36e-80a0f884fac0',
 '700f3645-f455-407c-8e4f-0f36f293faa6',
 '14d2aedf-3b5a-4616-9e0b-fc193c7f85f8',
 'd7a10d19-1a47-4ee3-8f21-b7753529573e',
 '3f1d5918-c871-4999-9d51-bfce59fe9aa0',
 '85d73a11-d17c-4b8c-b9f7-05083d172830',
 'db824eea-e3eb-4882-8e1d-9bc6c8f85dcb',
 '27a9b2c9-5771-4bc4-a3f7-250b2d078d92',
 '30ec303e-9cf6-4b6c-a646-1c6652643ed6',
 '7e242f12-5060-44d1-8238-2114475ffb5e',
 '074986ba-17e6-4686-9cb6-41c6ecac6d97',
 '15bf1417-ef41-4d06-a83c-f333f59c2898',
 'b8f670c3-501b-48a9-9e6f-4d2cdafd312b',
 '2ecfc104-5cb0-4382-90dd-06b8bb59684f',
 '5bd756eb-e277-40ed-9a05-6aec01b524c1',
 '48fe798a-b2e5-450e-a3e8-c86a5d0eb595',
 'a98d175a-6df5-4881-931d-e4d072ebd31e',
 '1eea4439-9991-45d3-bc74-e35c20f59473',
 '1914bef6-3ae9-4769-88ca-d53e201df453',
 '11d45db2-d565-4b56-afcf-47a4b883d69f',
 '69708268-c4bf-

## Testing

In [27]:
question = "Who is responsible for conducting the risk analysis and what methodology is used"

results = elastic_vector_search.similarity_search(question, 20)

In [28]:
len(results)

20

In [29]:
results[0].page_content

'Die Auditoren wählen für die Prüfung geeignete Methoden aus. Dies sind zum Beispiel Dokumentsichtung oder Interviews mit Anwendern, Administratoren'